In [117]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
# PyTorch related imports would go here
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
#from us import states
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re

In [118]:
SETTINGS_NO = "Case8"

In [119]:
# from google.colab import files

# # This will prompt you to upload a file from your local system
# uploaded = files.upload()
# filename = next(iter(uploaded))  # Get the first (and only) filename

# DATASET_NAME = "SPECIATE_V01282025_73_species.csv"
DATASET_NAME = "FinalDataset_transposed_02282025.csv"

df = pd.read_csv('PM25-Speciated/datasetasoffeb282025/'+DATASET_NAME, encoding='ISO-8859-1')

In [120]:
# #DATASET_NAME = "FinalDataset_transposed_02282025.csv"
# DATASET_NAME =filename
# df = pd.read_csv(DATASET_NAME, encoding='ISO-8859-1')

In [121]:
def clean_text(text):
    # Remove numbers and special characters
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [122]:
unique_scc1_values = df['SCC2'].unique()
pivot_df = df
pivot_df['SCC1'] = pd.factorize(pivot_df['SCC1'])[0]
pivot_df['SCC3'] = pd.factorize(pivot_df['SCC3'])[0]

In [123]:
# Step 2: Ensure SCC2 is treated as a string column
pivot_df['SCC2'] = pivot_df['SCC2'].astype(str)

# Split SCC2 based on ;
split_SCC2 = pivot_df['SCC2'].str.split(';', expand=True)

# Get unique sentence fragments
unique_fragments = pd.unique(split_SCC2.values.ravel('K'))

print(unique_fragments)

# Remove None values (if any)
unique_fragments = [fragment for fragment in unique_fragments if pd.notna(fragment) and fragment != 'nan']

# Create one-hot encoding for each fragment
one_hot_encoded = pd.DataFrame(0, index=pivot_df.index, columns=unique_fragments)

print(one_hot_encoded)

for col in split_SCC2.columns:
    for idx, fragment in split_SCC2[col].items():
        if pd.notna(fragment) and fragment != 'nan':
            one_hot_encoded.at[idx, fragment] = 1

# Step 3: Convert each row of one-hot encoding into a tuple
one_hot_tuples = [tuple(row) for row in one_hot_encoded.values]

# Step 4: Assign a unique sequential number to each unique combination
unique_combinations = pd.factorize(one_hot_tuples)[0]

# Add the sequential numbers to the DataFrame
pivot_df['SCC2'] = unique_combinations

# Display the transformed DataFrame
print(pivot_df)

['Miscellaneous' 'Electric Generation' 'Boiler' 'Agriculture' 'Road'
 'Mobile' 'Petrochemical' 'Biomass Burning' 'Waste' 'Geological' 'Desert'
 'Volcanic' 'Construction' 'Non-industrial' 'Industrial' 'Windblown'
 'Geothermal' 'Regional' 'Residential Heating' 'Cooking'
 'Waste Combustion' 'Mineral Products' 'Turbine' 'Process Heater'
 'Cigarette' 'Surface Coating' 'Pulp And Paper' 'Residential'
 'Stationary Engine' 'Furnace' 'Domestic' 'Chemical Manufacturing'
 'Combustion' 'Refinery' 'Incinerator' 'Secondary Aluminum Furnace'
 'Melting Pot' 'Smelting' 'Crop Heating' 'Food And Agriculture' None
 ' Boiler' ' Onroad' ' Catalytic Cracker' ' Residential Wood Combustion'
 ' Agriculture' ' Incinerator' ' Wildfire' ' Charbroiling' ' Kiln'
 ' Stirfrying' ' Process Heater' ' Frying' ' Spraying' ' Marine'
 ' Furnace' ' Aircraft' ' Metal' ' Silo' ' Gas Turbine' ' Prescribed Fire'
 ' Metal And Construction' ' Wood Products' ' Shredding' ' Mining'
 ' Calciner' ' Space Heater And Water Heater' ' Spac

In [124]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer

print(pivot_df.columns[3:])

# imputer = IterativeImputer()
#imputer = SimpleImputer(strategy='median')  #  option 1
imputer = KNNImputer() # option 2
df_imputed = pd.DataFrame(imputer.fit_transform(pivot_df.iloc[:,3:]), columns=df.columns[3:])
pivot_df[pivot_df.columns[3:]]=df_imputed

# # Opton 3 for minimum
# min_values = pivot_df.iloc[:, 3:].min()
# imputers = {col: SimpleImputer(strategy='constant', fill_value=min_val) for col, min_val in min_values.items()}
# # Apply imputation column by column
# for col in pivot_df.columns[3:]:
#     pivot_df[[col]] = imputers[col].fit_transform(pivot_df[[col]])

pivot_df



Index(['SCC1', 'SCC2', 'SCC3', 'Aluminum', 'Ammonium', 'Antimony', 'Arsenic',
       'Barium', 'Cadmium', 'Calcium', 'Chloride ion', 'Chromium', 'Cobalt',
       'Copper', 'Elemental Carbon', 'Indium', 'Iron', 'Lead', 'Magnesium',
       'Manganese', 'Mercury', 'Molybdenum', 'Nickel', 'Nitrate',
       'Organic carbon', 'Phosphorus', 'Potassium', 'Potassium ion',
       'Selenium', 'Silicon', 'Silver', 'Sodium', 'Sodium ion', 'Strontium',
       'Sulfate', 'Sulfur', 'Tin', 'Titanium', 'Vanadium', 'Zinc',
       'Zirconium'],
      dtype='object')


,Unnamed: 0,PROFILE_CODE,PROFILE_NAME,SCC1,SCC2,SCC3,Aluminum,Ammonium,Antimony,Arsenic,...,Sodium,Sodium ion,Strontium,Sulfate,Sulfur,Tin,Titanium,Vanadium,Zinc,Zirconium
0,0,2.5,Overall Composite,0.0,0.0,0.0,4.4430,0.022000,0.7650,1.47600,...,1.652000,0.40308,0.0140,5.5140,2.9720,0.1170,0.1940,0.06100,2.2330,0.0020
1,1,3190.0,Coal Combustion,1.0,1.0,1.0,5.9680,0.347600,0.0000,0.00000,...,0.952800,0.31000,0.1964,10.1716,2.9480,0.0000,0.4315,0.00000,0.0797,0.0247
2,2,3191.0,Coal Combustion,1.0,1.0,1.0,6.3904,0.335400,0.0000,0.00000,...,1.299540,0.40308,0.1373,2.1195,0.5934,0.0000,0.4018,0.00000,0.0857,0.0229
3,3,3192.0,Coal Combustion,1.0,1.0,1.0,5.5457,0.359800,0.0000,0.00000,...,0.724600,0.73522,0.2556,18.2238,5.3025,0.0000,0.4612,0.00000,0.0737,0.0266
4,4,3193.0,Coal Combustion,1.0,1.0,1.0,6.9752,0.341200,0.0000,0.00000,...,1.268000,0.40200,0.2784,7.5760,2.2323,0.0000,0.4532,0.00000,0.0653,0.0261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,1341,900122.5,Clay And Fly Ash Sintering - Average,5.0,81.0,0.0,0.4650,0.919540,1.5214,2.01520,...,0.520000,4.05756,0.0288,9.6000,3.2000,0.1512,0.0070,0.00400,0.0090,0.0030
1342,1342,900132.5,Mineral Products - Average,2.0,50.0,0.0,2.1110,2.388900,0.0210,0.00800,...,1.466000,2.86062,0.0120,14.0610,5.1430,0.0040,0.1340,0.00400,0.1270,0.0010
1343,1343,900142.5,Petroleum Industry - Average,7.0,27.0,0.0,2.4840,11.616820,0.0010,0.74308,...,3.773286,2.86062,0.0280,23.5000,10.0560,0.0010,0.0590,0.04716,0.2780,0.0250
1344,1344,900152.5,Pulp And Paper Industry,1.0,47.0,0.0,0.0900,4.842352,0.0040,0.00100,...,10.775000,3.47536,0.0070,20.3960,5.6700,0.0080,0.0030,0.00900,0.0170,0.0030


In [125]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter

In [126]:
def assign_cluster_name(cluster):
    names = pivot_df[pivot_df['speciated_cluster'] == cluster]['PROFILE_NAME']
    most_common_name = Counter(names).most_common(1)[0][0]
    return most_common_name

In [127]:
com_map = {}
loop_info = []
#parameters
StartCol=3
k_SVD = 16
K_knn = 5

In [128]:
#X = pivot_df.iloc[:, 3:]
## scaler = StandardScaler()
#scaler = MinMaxScaler()
#X = scaler.fit_transform(X)

In [129]:
#pip install cdlib

In [130]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from scipy.sparse.linalg import svds

import cdlib
from cdlib import algorithms
import networkx as nx

In [131]:
X = pivot_df.iloc[:, 6:]
X

,Aluminum,Ammonium,Antimony,Arsenic,Barium,Cadmium,Calcium,Chloride ion,Chromium,Cobalt,...,Sodium,Sodium ion,Strontium,Sulfate,Sulfur,Tin,Titanium,Vanadium,Zinc,Zirconium
0,4.4430,0.022000,0.7650,1.47600,0.0520,0.4210,2.6140,0.541160,0.1030,0.03100,...,1.652000,0.40308,0.0140,5.5140,2.9720,0.1170,0.1940,0.06100,2.2330,0.0020
1,5.9680,0.347600,0.0000,0.00000,1.3315,0.0000,3.4536,1.208760,0.0176,0.00000,...,0.952800,0.31000,0.1964,10.1716,2.9480,0.0000,0.4315,0.00000,0.0797,0.0247
2,6.3904,0.335400,0.0000,0.00000,0.3697,0.0000,2.5607,0.247380,0.0149,0.00000,...,1.299540,0.40308,0.1373,2.1195,0.5934,0.0000,0.4018,0.00000,0.0857,0.0229
3,5.5457,0.359800,0.0000,0.00000,2.2933,0.0000,4.3464,0.757140,0.0202,0.00000,...,0.724600,0.73522,0.2556,18.2238,5.3025,0.0000,0.4612,0.00000,0.0737,0.0266
4,6.9752,0.341200,0.0000,0.00000,1.2348,0.0000,4.6981,0.510020,0.0198,0.00000,...,1.268000,0.40200,0.2784,7.5760,2.2323,0.0000,0.4532,0.00000,0.0653,0.0261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,0.4650,0.919540,1.5214,2.01520,0.0790,0.5924,15.0700,2.895800,0.0040,0.04014,...,0.520000,4.05756,0.0288,9.6000,3.2000,0.1512,0.0070,0.00400,0.0090,0.0030
1342,2.1110,2.388900,0.0210,0.00800,0.0610,0.0150,9.8600,4.308860,0.1260,0.17500,...,1.466000,2.86062,0.0120,14.0610,5.1430,0.0040,0.1340,0.00400,0.1270,0.0010
1343,2.4840,11.616820,0.0010,0.74308,0.0000,0.0000,2.5180,4.210540,0.2750,1.00000,...,3.773286,2.86062,0.0280,23.5000,10.0560,0.0010,0.0590,0.04716,0.2780,0.0250
1344,0.0900,4.842352,0.0040,0.00100,0.0160,0.0030,0.4240,5.772707,0.0020,0.04932,...,10.775000,3.47536,0.0070,20.3960,5.6700,0.0080,0.0030,0.00900,0.0170,0.0030


In [132]:
folder = "PM25-Speciated/ClusterswoSCC/"
# folder = "PM25-Speciated/ClusterswSCC/"

In [133]:
# Assuming df and pivot_df are your dataframes loaded earlier

# Loop for different values of k_SVD, K_knn, and StartCol
k_SVD_values = [10, 15, 20,30]  # Example values for k_SVD
K_knn_values = [3, 4,5, 6,7,8,9]  # Example values for K_knn
# change this to 3 without SCC hotcoding data
StartCol_values = [6]#[3, 4,5, 6]  # Example start columns

AllGraphs=[]

for StartCol in StartCol_values:
    for k_SVD in k_SVD_values:
        for K_knn in K_knn_values:
            X = pivot_df.iloc[:, StartCol:]

            # Apply SVD to X
            U, S, VT = svds(X.values, k_SVD)
            X_svd = U @ np.diag(S) @ VT
            X = X_svd

            # Compute KNN
            knn = NearestNeighbors(n_neighbors=K_knn)  # You can change the number of neighbors
            knn.fit(X)
            distances, indices = knn.kneighbors(X)

            # Create a graph
            G = nx.Graph()

            # Add nodes with PROFILE_NAME as labels
            for i, profile_name in enumerate(df['PROFILE_NAME']):
                G.add_node(i, label=profile_name)

            # Add edges based on KNN
            for i, neighbors in enumerate(indices):
                for neighbor in neighbors:
                    if i != neighbor:  # Avoid self-loops
                        G.add_edge(i, neighbor, weight=1.0)
                        
            
            AllGraphs.append(G)
            

#             # Apply Louvain algorithm
#             coms = algorithms.louvain(G, weight='weight', resolution=1., randomize=False)
#             #print(coms)

#             for i in range(len(coms.communities)):
#                 community = coms.communities[i]
#                 for j in community:
#                     if j in com_map:
#                         com_map[j].append(i)
#                     else:
#                         com_map[j]=[i]
#             cluster_number = len(coms.communities)
#             #print(cluster_number)
#             print(f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}, No. of Clusters: {cluster_number} ")
#             print_string =f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}, No. of Clusters: {cluster_number} "
#             loop_info.append(print_string)

            print(f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}")
                        

Current StartCol: 6, Current k_SVD: 10, Current K_knn: 3
Current StartCol: 6, Current k_SVD: 10, Current K_knn: 4
Current StartCol: 6, Current k_SVD: 10, Current K_knn: 5
Current StartCol: 6, Current k_SVD: 10, Current K_knn: 6
Current StartCol: 6, Current k_SVD: 10, Current K_knn: 7
Current StartCol: 6, Current k_SVD: 10, Current K_knn: 8
Current StartCol: 6, Current k_SVD: 10, Current K_knn: 9
Current StartCol: 6, Current k_SVD: 15, Current K_knn: 3
Current StartCol: 6, Current k_SVD: 15, Current K_knn: 4
Current StartCol: 6, Current k_SVD: 15, Current K_knn: 5
Current StartCol: 6, Current k_SVD: 15, Current K_knn: 6
Current StartCol: 6, Current k_SVD: 15, Current K_knn: 7
Current StartCol: 6, Current k_SVD: 15, Current K_knn: 8
Current StartCol: 6, Current k_SVD: 15, Current K_knn: 9
Current StartCol: 6, Current k_SVD: 20, Current K_knn: 3
Current StartCol: 6, Current k_SVD: 20, Current K_knn: 4
Current StartCol: 6, Current k_SVD: 20, Current K_knn: 5
Current StartCol: 6, Current k_

In [134]:
AllGraphs

In [135]:
import networkx as nx
from networkx.utils import open_file
import itertools

In [136]:
def combine_graphs(AllGraphs):
    """
    Combine multiple graphs using nx.compose_all() and sum edge weights.
    
    Args:
        AllGraphs: List of networkx graphs with the same nodes
        
    Returns:
        A new graph with combined edge weights
    """
    if not AllGraphs:
        return nx.Graph()
    
    # Use compose_all to combine graphs
    combined = nx.compose_all(AllGraphs)
    
    # For edges that exist in multiple graphs, sum their weights
    for u, v, data in combined.edges(data=True):
        data['weight'] = sum(G.get_edge_data(u, v, {}).get('weight', 1) 
                          for G in AllGraphs if G.has_edge(u, v))
    
    return combined

combined_graph = combine_graphs(AllGraphs)

combined_graph

In [137]:
def print_edges_for_node_sorted(graph, node):
    if node not in graph:
        print(f"Node {node} not found in the graph")
        return
    
    edges = [(neighbor, data.get('weight', 1)) for neighbor, data in graph[node].items()]
    edges.sort(key=lambda x: x[1], reverse=True)  # Sort by weight descending
    
    print(f"Edges and weights for node {node} (sorted by weight):")
    for neighbor, weight in edges:
        print(f"({node}, {neighbor}): weight = {weight}")

# Example usage
print_edges_for_node_sorted(combined_graph, 0)

Edges and weights for node 0 (sorted by weight):
(0, 6): weight = 28.0
(0, 1345): weight = 28.0
(0, 193): weight = 23.0
(0, 71): weight = 22.0
(0, 2): weight = 22.0
(0, 841): weight = 20.0
(0, 4): weight = 17.0
(0, 1336): weight = 14.0
(0, 1225): weight = 12.0
(0, 1196): weight = 10.0
(0, 1331): weight = 10.0
(0, 814): weight = 9.0
(0, 805): weight = 8.0
(0, 1214): weight = 7.0
(0, 1338): weight = 5.0
(0, 205): weight = 4.0
(0, 1055): weight = 1.0


In [138]:
for g in AllGraphs:
    print("*"*100)
    print_edges_for_node_sorted(g,0)

****************************************************************************************************
Edges and weights for node 0 (sorted by weight):
(0, 6): weight = 1.0
(0, 805): weight = 1.0
(0, 71): weight = 1.0
(0, 841): weight = 1.0
(0, 1345): weight = 1.0
****************************************************************************************************
Edges and weights for node 0 (sorted by weight):
(0, 6): weight = 1.0
(0, 805): weight = 1.0
(0, 1196): weight = 1.0
(0, 71): weight = 1.0
(0, 841): weight = 1.0
(0, 1345): weight = 1.0
****************************************************************************************************
Edges and weights for node 0 (sorted by weight):
(0, 6): weight = 1.0
(0, 805): weight = 1.0
(0, 1196): weight = 1.0
(0, 841): weight = 1.0
(0, 71): weight = 1.0
(0, 193): weight = 1.0
(0, 1345): weight = 1.0
****************************************************************************************************
Edges and weights for node 0 (sorted by

In [139]:
@open_file(1, mode='wb')
def save_mm(graph, path):
    """
    Save graph in Matrix Market format with edge weights.
    
    Args:
        graph: NetworkX graph to save
        path: File path to save to
    """
    # Get the number of nodes and edges
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    
    # Create a mapping from nodes to integer indices
    node_index = {node: i+1 for i, node in enumerate(graph.nodes())}  # Matrix Market uses 1-based indexing
    
    # Write the header
    header = f"%%MatrixMarket matrix coordinate real symmetric\n%\n{num_nodes} {num_nodes} {num_edges}\n"
    path.write(header.encode('ascii'))
    
    # Write the edges
    for u, v, data in graph.edges(data=True):
        weight = data.get('weight', 1)
        path.write(f"{node_index[u]} {node_index[v]} {weight}\n".encode('ascii'))

# Example usage:
# Assuming AllGraphs is your list of graphs
save_mm(combined_graph, folder+"concensus-withoutSCC.mtx")

In [140]:
filename = folder+"concensus-withoutSCC.gexf"
print("Writing gephi...."+filename)
nx.write_gexf(combined_graph, filename)
print("Done....")

Writing gephi....PM25-Speciated/ClusterswoSCC/concensus-withoutSCC.gexf
Done....


## Apply Louvain algorithm

In [141]:
fileno = 0

for w in [True, False]:
    
    print("weighted" if w else "unweighted")

    for resolution in [0.3, 0.5, 0.8, 1.0, 1.5, 2.0, 2.5, 3.0]: #default 1.0
        
        if w:
            coms = algorithms.louvain(combined_graph, weight='weight', resolution=resolution, randomize=False)
        else:
            coms = algorithms.louvain(combined_graph, resolution=resolution, randomize=False)
        #print(coms)
        
        # File path where you want to save the clusters
        output_file = folder+"cluster_no_"+str(fileno)+".txt"

        # Write clusters to the file
        with open(output_file, 'w') as f:
            for cluster in coms.communities:
                # Convert each node number to string and join with spaces
                line = ' '.join(map(str, cluster))
                f.write(line + '\n')

        print(f"Clusters written to {output_file}")

#         for i in range(len(coms.communities)):
#             community = coms.communities[i]
#             for j in community:
#                 if j in com_map:
#                     com_map[j].append(i)
#                 else:
#                     com_map[j]=[i]
        cluster_number = len(coms.communities)
        print(f"No. of Clusters: {cluster_number} ")
        fileno+=1

weighted
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_0.txt
No. of Clusters: 115 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_1.txt
No. of Clusters: 24 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_2.txt
No. of Clusters: 16 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_3.txt
No. of Clusters: 17 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_4.txt
No. of Clusters: 25 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_5.txt
No. of Clusters: 32 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_6.txt
No. of Clusters: 36 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_7.txt
No. of Clusters: 40 
unweighted
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_8.txt
No. of Clusters: 115 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_9.txt
No. of Clusters: 24 
Clusters written to PM25-Speciated/ClusterswoSCC/cluster_no_10.txt
No. of Clusters: 16 
Clusters written to 

In [142]:
# import csv
# # Writing com_map to CSV
# with open('com_map_KNNImputer_wSCC.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['Node', 'Communities'])
#     for key, value in com_map.items():
#         writer.writerow([key, ', '.join(map(str, value))])


In [143]:
# with open('loop_info_KNNImputer_wSCC.txt', mode='w') as file:
#     for line in loop_info:
#         file.write(line + '\n')

In [144]:
# com_map